# 이상치(Outlier) 탐지 및 처리: IQR 규칙 활용하기

이 노트북은 데이터 전처리 과정에서 중요한 단계 중 하나인 **이상치(Outlier)**를 탐지하고 처리하는 방법을 다룹니다.

### 이상치란?
이상치란 대부분의 데이터 값들과는 동떨어진, 극단적으로 크거나 작은 값을 의미합니다. (예: 직원 연봉 데이터에 포함된 CEO의 연봉) 이러한 값들은 데이터의 평균을 왜곡하고, 머신러닝 모델의 학습을 방해할 수 있어 적절한 처리가 필요합니다.

**학습 목표:**
1.  `Boxplot`을 사용하여 이상치를 시각적으로 확인합니다.
2.  **IQR(사분위수 범위, Interquartile Range)** 규칙을 사용하여 이상치를 수학적으로 정의하고 탐지합니다.
3.  탐지된 이상치를 경계값으로 대체(Capping)하는 방법을 알아봅니다.

### 1. 라이브러리 임포트 및 샘플 데이터 생성

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 결측치(NaN)를 포함한 샘플 데이터 생성
data = pd.Series([1, 2, np.nan, 4, 5, 6, np.nan, 8, 9, 10])

# 결측치를 100이라는 극단적인 값으로 대체하여 이상치를 만듭니다.
data[data.isna()] = 100

print("이상치가 포함된 데이터:")
print(data.values)

### 2. Boxplot으로 이상치 시각적 확인 (처리 전)

Boxplot은 데이터의 분포를 시각적으로 보여주는 강력한 도구입니다. 상자 바깥쪽에 그려지는 점(o)들은 이상치를 의미합니다.

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(data, vert=True) # vert=True: 수직, False: 수평
plt.title("Boxplot of Data with Outliers")
plt.ylabel("Value")
plt.show()
# 100이라는 값이 다른 데이터들과 동떨어져 이상치로 표시된 것을 볼 수 있습니다.

### 3. IQR 규칙으로 이상치 탐지

가장 널리 사용되는 이상치 탐지 규칙 중 하나는 IQR(사분위수 범위)을 이용하는 것입니다.

1.  **Q1**: 데이터의 25% 지점에 있는 값 (1사분위수)
2.  **Q3**: 데이터의 75% 지점에 있는 값 (3사분위수)
3.  **IQR**: `Q3 - Q1`
4.  **정상 범위**: `Q1 - 1.5 * IQR` (하한) 부터 `Q3 + 1.5 * IQR` (상한) 까지

이 범위를 벗어나는 모든 값은 이상치로 간주합니다.

In [ ]:
def find_outlier_boundaries(data):
    """IQR 규칙에 따라 이상치의 하한과 상한 경계를 반환합니다."""
    # 1사분위수(Q1)와 3사분위수(Q3) 계산
    q1, q3 = np.percentile(data, [25, 75])
    
    # IQR 계산
    iqr = q3 - q1
    
    # 하한 및 상한 경계 계산
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    
    return lower_bound, upper_bound

# 실제 데이터에 함수를 적용하여 경계값 찾기
lower, upper = find_outlier_boundaries(data)
print(f"이상치 하한 경계: {lower:.2f}")
print(f"이상치 상한 경계: {upper:.2f}")

### 4. 이상치 처리 (Capping)

탐지된 이상치를 처리하는 방법에는 여러 가지가 있습니다 (제거, 평균/중앙값으로 대체 등). 여기서는 **Capping** 방법을 사용하여, 이상치를 위에서 계산한 하한/상한 경계값으로 대체해 보겠습니다.

In [ ]:
# 원본 데이터를 복사하여 처리
data_capped = data.copy()

# 하한보다 작은 값은 하한값으로, 상한보다 큰 값은 상한값으로 대체
data_capped[data_capped < lower] = lower
data_capped[data_capped > upper] = upper

print("이상치 처리 후 데이터:")
print(data_capped.values)

### 5. Boxplot으로 결과 확인 (처리 후)

이상치가 처리된 데이터로 다시 Boxplot을 그려보면, 더 이상 상자 바깥쪽에 점이 나타나지 않는 것을 확인할 수 있습니다.

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(data_capped, vert=True)
plt.title("Boxplot of Data after Capping Outliers")
plt.ylabel("Value")
plt.show()